###README</br>
**Q2**</br>
**Liang Zhao 260781081**
- Every question is listed in the table of contents.
- Some questions are answered together in one code snippet.
- This colab note should be run in GPU in environment.

In [1]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

import numpy as np
import cv2
import os
import matplotlib.pyplot as plt

#2.Image Classification with Convolution Neural Network (CNN)

##2.1. Use Pytorch class torchvision.datasets.MNIST to download the dataset. Use batch size of 32.

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5), (0.5))])

trainset = torchvision.datasets.MNIST(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.MNIST(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


/usr/local/lib/python3.6/dist-packages/torchvision/datasets/mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


##2.2. Implement a CNN with the layers mentioned below.

In [3]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        # A convolution layer with 32 kernels of size 3×3.
        self.conv1 = nn.Conv2d(1, 32, 3)
        # A convolution layer with 64 kernels of size 3×3.
        self.conv2 = nn.Conv2d(32, 64, 3)
        # A maxpool layer with kernels of size 2×2.
        self.pool = nn.MaxPool2d(2, 2)
        # A convolution layer with 64 kernels of size 3×3.
        self.conv3 = nn.Conv2d(64, 64, 3)
        # A convolution layer with 64 kernels of size 3×3.
        self.conv4 = nn.Conv2d(64, 64, 3)
        # A Linear layer with output size of 10.
        self.fc1 = nn.Linear(4096, 10)


    def forward(self, x):
        # ReLU activations.
        x = F.relu(self.conv1(x))
        x = self.pool(F.relu(self.conv2(x)))
        x = F.relu(self.conv3(x))
        x = F.relu(self.conv4(x))
        # A flattening layer.
        x = x.view(-1, 4096)
        x = self.fc1(x)
        return x

net = Net()

##2.3. Create an instance of SGD optimizer with learning rate of 0.001. Use the default setting for rest of the hyperparameters. Create an instance of categorical cross entropy criterion. 

##2.4. Train the CNN for 10 epochs.

##2.5. Predicts labels of the test images using the above trained CNN. Measure and display classification accuracy.

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
net = Net()
net = net.to(device)

#2.3 Create an instance of categorical cross entropy criterion
criterion = nn.CrossEntropyLoss()
#2.3 Create an instance of SGD optimizer
optimizer = optim.SGD(net.parameters(), lr=0.001)

#2.4. Train the CNN for 10 epochs.
for epoch in range(10):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = inputs.to(device)
        labels = labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.item()
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')

#2.5. Predicts labels of the test images
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        images, labels = data
        images = images.to(device)
        labels = labels.to(device)
        outputs = net(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the test images: %d %%' % (
    100 * correct / total))

Finished Training
Accuracy of the network on the test images: 96 %
